# Histogram based Gradient Boosting

## Import required libraries

In [1]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

## Load dataset and discretize popularity

In [2]:
df = pd.read_csv('../../Dataset/data_merged.csv')
mean_popularity = 42
df["popularity"] = [ 1 if i >= mean_popularity else 0 for i in df.popularity ]
df["popularity"].head()

0    0
1    0
2    0
3    0
4    0
Name: popularity, dtype: int64

## Split features, labels and train-test data

In [3]:
X  = df.drop(['popularity'], axis=1)
y = df['popularity']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=21)

## Histogram based gradient-boosting

In [88]:
gb_clf = HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000)

In [89]:
gb_clf.fit(X_train, y_train)

HistGradientBoostingClassifier(loss='binary_crossentropy', max_iter=1000)

In [90]:
gb_clf.score(X_train, y_train)

0.8991247169482669

In [91]:
gb_clf.score(X_test, y_test)

0.8895953085989665

In [92]:
y_pred = gb_clf.predict(X_test)

In [93]:
print(confusion_matrix(y_test, y_pred))
print("Classification report\n")
print(classification_report(y_test, y_pred))

[[24345  1086]
 [ 2717  6298]]
Classification report

              precision    recall  f1-score   support

           0       0.90      0.96      0.93     25431
           1       0.85      0.70      0.77      9015

    accuracy                           0.89     34446
   macro avg       0.88      0.83      0.85     34446
weighted avg       0.89      0.89      0.89     34446



## Bayesian Search

In [5]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_objective
from matplotlib import pyplot as plt

In [6]:
opt = BayesSearchCV(
    HistGradientBoostingClassifier(random_state=42),
    {
        'max_iter' : Integer(100, 200),
        'learning_rate' : Real(0.01, 1),
        'max_leaf_nodes' : Integer(10,100),
        'min_samples_leaf' : Integer(10,100),
#         'C': (1e-6, 1e+6, 'log-uniform'),
#         'gamma': (1e-6, 1e+1, 'log-uniform'),
#         'degree': (1, 8),  # integer valued parameter
#         'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=128,
    cv=3
)

In [ ]:
opt.fit(X_train, y_train)

from joblib import dump
dump(opt, "Saved models/GradBoost")

## Best parameters and scores obtained

In [97]:
print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))
print("best params: %s" % str(opt.best_params_))

val. score: 0.89182343378041
test score: 0.8911049178424201
best params: OrderedDict([('learning_rate', 0.060833085502393476), ('max_iter', 200), ('max_leaf_nodes', 100), ('min_samples_leaf', 78)])


In [98]:
y_pred = opt.predict(X_test)

In [99]:
print(confusion_matrix(y_test, y_pred))
print("Classification report\n")
print(classification_report(y_test, y_pred))

[[24379  1052]
 [ 2699  6316]]
Classification report

              precision    recall  f1-score   support

           0       0.90      0.96      0.93     25431
           1       0.86      0.70      0.77      9015

    accuracy                           0.89     34446
   macro avg       0.88      0.83      0.85     34446
weighted avg       0.89      0.89      0.89     34446



On increasing number of iterations, max_leaf_nodes, min_samples_leaf etc., learning rate automatically goes down and results remain the same more or less.

## Visualize results with partial dependence plots

In [100]:
# plot_objective(opt.optimizer_results_[0],
#                    dimensions=["n_estimators", "learning_rate"],
#                    n_minimum_search=int(1e8))